In [37]:
# execute this cell before you start

import tensorflow as tf
from tensorflow.keras import layers

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras

print(tf.VERSION)
print(tf.keras.__version__)


1.12.0
2.1.6-tf


#  CA4
## due on 02/04/2019

to submit the assignment, please do the following:

- do `Cell -> All output -> Clear` to clear all your output
- save the notebook (CA3.ipynb)

# The Boston Housing Data

Consider the data in  `keras.datasets.boston_housing`. In this case, there are only about 400 training datasets, where each dataset consists of 13 input values which are characteristic for a given property. The output corresponds to the property price. The meaning of the various columns is explained in https://www.kaggle.com/c/boston-housing.

In contrast to the previous examples, which were categorisation problems, this is now a regression problem. The challenge is to train a network, which is able to predict the price of the property. 

You will again find lots of examples on the internet, and it is okay to use inspiration as long as you provide the source. 

Adhere to the following rules:

- Train the network on the logarithm of the price, not on the price itself. Explain why this makes sense. 
- You will find many examples, which use `sci-kit learn` or other packages, which we did not do in the course. Do not use them, and restrict yourself to methods and libraries which we covered
- Try to find a network, which has the smallest amount of trainable parameters, while still providing good predictions of the price.  Discuss, how small you can go. 
- Once you have trained the network, explore the correlations which this network predicts:
    - Which inputs have a positive price correlation? 
    - Which inputs have a negative price correlation? 
    - Which inputs have little/no influence on the price?
    
  Investigate this by feeding into the network some artificial data, which you obtain from the testing data by varying one of the input columns.
  
  
Optional challenge (no extra points but extra insight!):

Compare the results with standard regression methods, for example as in ST4060/ST4061 in case you have covered them. 



In [38]:
(train_x, train_y), (test_x, test_y) = keras.datasets.boston_housing.load_data()
log_train_y = np.log(train_y)
log_test_y = np.log(test_y)

# from https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html
var_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE',
             'DIS', 'RAD', 'TAX', 'PTRATIO', 'BLACK', 'LSTAT', 'MEDV']

Method to print first n entries in the training data

In [ ]:
def printFirstN(n):
    print(("{:8}"*len(var_names)).format(*var_names))
    for i in range(10):
        print(("{:<8.4}"*len(train_x[i])).format(*train_x[i]),
         "{:<8.3}".format(train_y[i]))

In [39]:
#Explore the data
print('Train X shape', train_x.shape)
print('Train Y shape', train_y.shape)
print('Test X shape', test_x.shape)
print('Train Y shape', test_y.shape)
print()

#inspect a few elements to get an idea of the data

printFirstN(10)

    

Train X shape (404, 13)
Train Y shape (404,)
Test X shape (102, 13)
Train Y shape (102,)

CRIM    ZN      INDUS   CHAS    NOX     RM      AGE     DIS     RAD     TAX     PTRATIO BLACK   LSTAT   MEDV    
1.232   0.0     8.14    0.0     0.538   6.142   91.7    3.977   4.0     307.0   21.0    396.9   18.72    15.2    
0.02177 82.5    2.03    0.0     0.415   7.61    15.7    6.27    2.0     348.0   14.7    395.4   3.11     42.3    
4.898   0.0     18.1    0.0     0.631   4.97    100.0   1.333   24.0    666.0   20.2    375.5   3.26     50.0    
0.03961 0.0     5.19    0.0     0.515   6.037   34.5    5.985   5.0     224.0   20.2    396.9   8.01     21.1    
3.693   0.0     18.1    0.0     0.713   6.376   88.4    2.567   24.0    666.0   20.2    391.4   14.65    17.7    
0.2839  0.0     7.38    0.0     0.493   5.708   74.3    4.721   5.0     287.0   19.6    391.1   11.74    18.5    
9.187   0.0     18.1    0.0     0.7     5.536   100.0   1.58    24.0    666.0   20.2    396.9   23.6     11.3    

It looks like the values of each predictor are very different in scale when compared to each other.
This may lead to difficulties in building a good enough model.
Hence scale them uniformly.

In [40]:
train_x = (train_x - train_x.min(axis=0)) / (train_x.max(axis=0) - train_x.min(axis=0))
test_x = (test_x - test_x.min(axis=0)) / (test_x.max(axis=0) - test_x.min(axis=0))

In [41]:
printFirstN(10)

CRIM    ZN      INDUS   CHAS    NOX     RM      AGE     DIS     RAD     TAX     PTRATIO BLACK   LSTAT   MEDV    
0.01378 0.0     0.2815  0.0     0.3148  0.4998  0.9145  0.2972  0.1304  0.2275  0.8936  1.0     0.4688   15.2    
0.00017370.825   0.05755 0.0     0.06173 0.7841  0.1318  0.5365  0.04348 0.3059  0.2234  0.9962  0.03808  42.3    
0.05498 0.0     0.6466  0.0     0.5062  0.2729  1.0     0.02118 1.0     0.914   0.8085  0.9461  0.04222  50.0    
0.00037420.0     0.1734  0.0     0.2675  0.4795  0.3254  0.5068  0.1739  0.06883 0.8085  1.0     0.1733   21.1    
0.04144 0.0     0.6466  0.0     0.6749  0.5451  0.8805  0.15    1.0     0.914   0.8085  0.9862  0.3565   17.7    
0.00312 0.0     0.2537  0.0     0.2222  0.4158  0.7353  0.3749  0.1739  0.1893  0.7447  0.9855  0.2762   18.5    
0.1032  0.0     0.6466  0.0     0.6481  0.3825  1.0     0.04705 1.0     0.914   0.8085  1.0     0.6035   11.3    
0.04598 0.0     0.7009  0.0     1.0     0.3693  1.0     0.02946 0.1739  0.4111  0.2234 

In [42]:
# mean = train_x.mean(axis=0)
# train_x = train_x - mean
# std = train_x.std(axis=0)
# train_x = train_x/std

# test_x = test_x-mean
# test_x = test_x/std

In [43]:
model = keras.models.Sequential()
model.add(keras.layers.Dense(128, activation=tf.nn.relu, input_shape=(train_x.shape[1],)))
model.add(keras.layers.Dense(128, activation=tf.nn.relu))
model.add(keras.layers.Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

model.fit(train_x, train_y, epochs=100, batch_size=1)


Epoch 1/100
404/404 [==============================] - 1s 1ms/step - loss: 139.6545 - mean_absolute_error: 8.1735
Epoch 2/100
404/404 [==============================] - 0s 864us/step - loss: 39.9601 - mean_absolute_error: 4.3182
Epoch 3/100
404/404 [==============================] - 0s 781us/step - loss: 26.9119 - mean_absolute_error: 3.6592
Epoch 4/100
404/404 [==============================] - 0s 809us/step - loss: 23.8190 - mean_absolute_error: 3.3822
Epoch 5/100
404/404 [==============================] - 0s 796us/step - loss: 20.5363 - mean_absolute_error: 3.1774
Epoch 6/100
404/404 [==============================] - 0s 801us/step - loss: 19.5024 - mean_absolute_error: 3.1044
Epoch 7/100
404/404 [==============================] - 0s 807us/step - loss: 17.6991 - mean_absolute_error: 3.0276
Epoch 8/100
404/404 [==============================] - 0s 814us/step - loss: 16.9792 - mean_absolute_error: 2.9201
Epoch 9/100
404/404 [==============================] - 0s 777us/step - loss: 15.2

404/404 [==============================] - 0s 850us/step - loss: 5.4451 - mean_absolute_error: 1.6724
Epoch 73/100
404/404 [==============================] - 0s 773us/step - loss: 5.8933 - mean_absolute_error: 1.7186
Epoch 74/100
404/404 [==============================] - 0s 789us/step - loss: 5.4379 - mean_absolute_error: 1.6820
Epoch 75/100
404/404 [==============================] - 0s 800us/step - loss: 5.9948 - mean_absolute_error: 1.7385
Epoch 76/100
404/404 [==============================] - 0s 739us/step - loss: 5.3022 - mean_absolute_error: 1.6988
Epoch 77/100
404/404 [==============================] - 0s 781us/step - loss: 5.9018 - mean_absolute_error: 1.7091
Epoch 78/100
404/404 [==============================] - 0s 807us/step - loss: 4.9832 - mean_absolute_error: 1.6438
Epoch 79/100
404/404 [==============================] - 0s 819us/step - loss: 5.6275 - mean_absolute_error: 1.7015
Epoch 80/100
404/404 [==============================] - 0s 802us/step - loss: 5.3511 - mean_a

In [44]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
